In [2]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [3]:
df = pd.read_csv('IMDB.csv')
df = df.sample(500)
df.to_csv('data.csv', index=False)
df.head()

,review,sentiment
601,I suppose if you like pure action... you'll fi...,negative
288,I just got done watching The Edge of Love (by ...,negative
7,Awesome Movie! Great combination of talents! I...,positive
327,This has to be the worst movie i've seen this ...,negative
46,"My parents may enjoy this show, but I fail to ...",negative


In [7]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [11]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
df = normalize_text(df)
df.head()

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ANIKET\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ANIKET\AppData\Roaming\nltk_data...
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\ANIKET\AppData\Roaming\nltk_data...


,review,sentiment
601,suppose like pure action find here suppose fin...,negative
288,got done watching edge love by way one worst t...,negative
7,awesome movie great combination talent huge fa...,positive
327,worst movie seen year watch lot cable plot rid...,negative
46,parent may enjoy show fail find humor it funny...,negative


In [12]:
df['sentiment'].value_counts()

sentiment
negative    252
positive    248
Name: count, dtype: int64

In [13]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [14]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

,review,sentiment
601,suppose like pure action find here suppose fin...,0
288,got done watching edge love by way one worst t...,0
7,awesome movie great combination talent huge fa...,1
327,worst movie seen year watch lot cable plot rid...,0
46,parent may enjoy show fail find humor it funny...,0


In [15]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [16]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [20]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/aniketsen123/Capstone-Project-.mlflow')
dagshub.init(repo_owner='aniketsen123', repo_name='Capstone-Project-', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


Initialized MLflow to track repo "aniketsen123/Capstone-Project-"

Repository aniketsen123/Capstone-Project- initialized!

<Experiment: artifact_location='mlflow-artifacts:/3854b8c799c3469c90d9a40d9e1431c8', creation_time=1762053541712, experiment_id='0', last_update_time=1762053541712, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [21]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model")

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-11-02 08:50:13,718 - INFO - Starting MLflow run...
2025-11-02 08:50:14,533 - INFO - Logging preprocessing parameters...
2025-11-02 08:50:15,961 - INFO - Initializing Logistic Regression model...
2025-11-02 08:50:15,962 - INFO - Fitting the model...
2025-11-02 08:50:15,995 - INFO - Model training complete.
2025-11-02 08:50:15,996 - INFO - Logging model parameters...
2025-11-02 08:50:16,471 - INFO - Making predictions...
2025-11-02 08:50:16,473 - INFO - Calculating evaluation metrics...
2025-11-02 08:50:16,484 - INFO - Logging evaluation metrics...
2025-11-02 08:50:18,136 - INFO - Saving and logging the model...
2025/11/02 08:50:18 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025-11-02 08:50:19,288 - ERROR - An error occurred: INTERNAL_ERROR: Response: {'error': 'unsupported endpoint, please contact support@dagshub.com'}
Traceback (most recent call last):
  File "C:\Users\ANIKET\AppData\Local\Temp\ipykernel_3192\2979654972.py", line 48, in 

🏃 View run adventurous-slug-549 at: https://dagshub.com/aniketsen123/Capstone-Project-.mlflow/#/experiments/0/runs/dc758dd0cd6d4c6a9e109a0dc286eb80
🧪 View experiment at: https://dagshub.com/aniketsen123/Capstone-Project-.mlflow/#/experiments/0
